In [1]:
import pandas as pd
import numpy as np
import os

In [1]:
import pandas as pd

df = pd.read_csv(r'C:\Users\ilyas\Downloads\customer_signups.csv')

print(df.head())

  customer_id             name                 email signup_date     source  \
0   CUST00000    Joshua Bryant                   NaN         NaN  Instagram   
1   CUST00001   Nicole Stewart   nicole1@example.com    02-01-24   LinkedIn   
2   CUST00002     Rachel Allen   rachel2@example.com    03-01-24     Google   
3   CUST00003  Zachary Sanchez  zachary3@mailhub.org    04-01-24    YouTube   
4   CUST00004              NaN  matthew4@mailhub.org    05-01-24   LinkedIn   

  region plan_selected marketing_opt_in age      gender  
0    NaN         basic               No  34      Female  
1   West         basic              Yes  29        Male  
2  North       PREMIUM              Yes  34  Non-Binary  
3    NaN           Pro               No  40        Male  
4   West       Premium               No  25       Other  


In [2]:
df_copy = df.copy()

df_copy['status'] = 'active'

In [3]:
print(df_copy.to_string())

    customer_id                    name                        email        signup_date     source   region plan_selected marketing_opt_in      age      gender  status
0     CUST00000           Joshua Bryant                          NaN                NaN  Instagram      NaN         basic               No       34      Female  active
1     CUST00001          Nicole Stewart          nicole1@example.com           02-01-24   LinkedIn     West         basic              Yes       29        Male  active
2     CUST00002            Rachel Allen          rachel2@example.com           03-01-24     Google    North       PREMIUM              Yes       34  Non-Binary  active
3     CUST00003         Zachary Sanchez         zachary3@mailhub.org           04-01-24    YouTube      NaN           Pro               No       40        Male  active
4     CUST00004                     NaN         matthew4@mailhub.org           05-01-24   LinkedIn     West       Premium               No       25       Other 

# Task 1 Loading & Cleaning Data


In [4]:
print("\n Missing values per column:")
print(df_copy.isnull().sum())


 Missing values per column:
customer_id          2
name                 9
email               34
signup_date          2
source               9
region              30
plan_selected        8
marketing_opt_in    10
age                 12
gender               8
status               0
dtype: int64


In [6]:
print("Column names and data types:")
print(df_copy.dtypes)

Column names and data types:
customer_id         object
name                object
email               object
signup_date         object
source              object
region              object
plan_selected       object
marketing_opt_in    object
age                 object
gender              object
status              object
dtype: object


In [7]:
df_copy['signup_date']=pd.to_datetime(df_copy['signup_date'], format='%d-%m-%y', errors='coerce')

In [8]:
df_copy['gender'] = df_copy['gender'].str.capitalize()
df_copy['plan_selected'] = df_copy['plan_selected'].str.capitalize()

In [9]:
df = df_copy.drop_duplicates(subset='customer_id', keep='first')

In [10]:
obj_cols = df_copy.select_dtypes(include='object').columns
df_copy[obj_cols] = df_copy[obj_cols].replace('', 'Unknown').fillna('Unknown')

# Task 2 Data Quality and Summary


In [5]:
unknown_counts = df_copy.isnull().sum()
print(unknown_counts)

customer_id          2
name                 9
email               34
signup_date          2
source               9
region              30
plan_selected        8
marketing_opt_in    10
age                 12
gender               8
status               0
dtype: int64


In [12]:
total_unknown_percentage = (df_copy.isnull().sum()).sum() / df_copy.size * 100
print(total_unknown_percentage)


0.21212121212121215


In [13]:
print("Number of duplicates removed:", df_copy.shape[0] - df_copy.drop_duplicates(subset='customer_id', keep='first').shape[0])
df_copy = df_copy.drop_duplicates(subset='customer_id', keep='first')

Number of duplicates removed: 1


In [14]:
changes = (df != df_copy).sum().sum()
print(f"Total changed cells: {changes}")

Total changed cells: 128


# Task 3 Summary Outputs (Using Pandas Aggrega ons)

In [15]:
signups_per_week = (
    df_copy
    .groupby(pd.Grouper(key='signup_date', freq='W'))['customer_id']
    .count()
    .reset_index()
    .rename(columns={'customer_id': 'signups_per_week'})
)
print(signups_per_week) 

   signup_date  signups_per_week
0   2024-01-07                 6
1   2024-01-14                 7
2   2024-01-21                 7
3   2024-01-28                 7
4   2024-02-04                 7
5   2024-02-11                 7
6   2024-02-18                 7
7   2024-02-25                 7
8   2024-03-03                 7
9   2024-03-10                 7
10  2024-03-17                 7
11  2024-03-24                 6
12  2024-03-31                 6
13  2024-04-07                 7
14  2024-04-14                 7
15  2024-04-21                 7
16  2024-04-28                 7
17  2024-05-05                 6
18  2024-05-12                 7
19  2024-05-19                 7
20  2024-05-26                 7
21  2024-06-02                 7
22  2024-06-09                 6
23  2024-06-16                 7
24  2024-06-23                 7
25  2024-06-30                 7
26  2024-07-07                 7
27  2024-07-14                 7
28  2024-07-21                 6
29  2024-0

In [16]:
signups_summary = (
    df_copy
    .groupby(['source', 'region', 'plan_selected'])['customer_id']
    .count()
    .reset_index()
    .rename(columns={'customer_id': 'signups_count'})
)
print(signups_summary.to_string(index=False))


   source  region plan_selected  signups_count
       ?? Central           Pro              1
       ??    East           Pro              1
       ??   South           Pro              1
       ??    West         Basic              1
       ??    West       Premium              1
       ??    West           Pro              1
 Facebook Central       Premium              4
 Facebook Central           Pro              3
 Facebook    East         Basic              3
 Facebook    East       Premium              3
 Facebook    East           Pro              4
 Facebook   North         Basic              1
 Facebook   North       Premium              2
 Facebook   North           Pro              4
 Facebook   South         Basic              2
 Facebook   South       Premium              4
 Facebook   South           Pro              1
 Facebook Unknown         Basic              1
 Facebook Unknown       Premium              2
 Facebook Unknown           Pro              4
 Facebook    

In [17]:
marketing_opt_in_counts = (
    df_copy[df_copy['marketing_opt_in'] == True]  
    .groupby('gender')['customer_id']
    .count()
    .reset_index()
    .rename(columns={'customer_id': 'opt_in_count'})
)
print(marketing_opt_in_counts.to_string(index=False))

Empty DataFrame
Columns: [gender, opt_in_count]
Index: []


In [18]:
df_copy['age'] = pd.to_numeric(df_copy['age'], errors='coerce')

df_clean = df_copy[(df_copy['age'] >= 10) & (df_copy['age'] <= 100)]

min_age = int(df_clean['age'].min())
max_age = int(df_clean['age'].max())
mean_age = int(round(df_clean['age'].mean()))
median_age = int(df_clean['age'].median())
null_count = int(df_clean['age'].isnull().sum())

age_classification = {
    'min_age': min_age,
    'max_age': max_age,
    'age_range_years': max_age - min_age,
    'mean_age': mean_age,
    'median_age': median_age,
    'null_count': null_count
}

print(age_classification)

{'min_age': 21, 'max_age': 60, 'age_range_years': 39, 'mean_age': 36, 'median_age': 34, 'null_count': 0}


### Task 4 will be in the coversheet

### Task 5 

In [19]:
import pandas as pd

df1 = pd.read_csv(r'C:\Users\ilyas\Downloads\support_tickets.csv')

print(df1.head())

   ticket_id customer_id ticket_date       issue_type resolved
0  TKT0000-1   CUST00203  2024-08-17          Billing      Yes
1  TKT0000-2   CUST00203  2024-07-22  Technical Error      Yes
2  TKT0000-3   CUST00203  2024-07-22            Other      Yes
3  TKT0001-1   CUST00266  2024-09-26    Account Setup      Yes
4  TKT0001-2   CUST00266  2024-10-09  Technical Error       No


In [20]:
df_copy1 = df1.copy()

df_copy1['status'] = 'active'

In [21]:
print(df_copy1.to_string())

     ticket_id customer_id ticket_date       issue_type resolved  status
0    TKT0000-1   CUST00203  2024-08-17          Billing      Yes  active
1    TKT0000-2   CUST00203  2024-07-22  Technical Error      Yes  active
2    TKT0000-3   CUST00203  2024-07-22            Other      Yes  active
3    TKT0001-1   CUST00266  2024-09-26    Account Setup      Yes  active
4    TKT0001-2   CUST00266  2024-10-09  Technical Error       No  active
5    TKT0001-3   CUST00266  2024-10-21            Other       No  active
6    TKT0002-1   CUST00152  2024-01-10            Other      Yes  active
7    TKT0003-1   CUST00009  2024-10-03  Technical Error       No  active
8    TKT0004-1   CUST00233  2024-09-18            Other      Yes  active
9    TKT0004-2   CUST00233  2024-08-21    Account Setup      Yes  active
10   TKT0004-3   CUST00233  2024-09-14          Billing       No  active
11   TKT0005-1   CUST00226  2024-08-16          Billing      Yes  active
12   TKT0005-2   CUST00226  2024-09-04          Bil

In [22]:
merged = pd.merge(df_copy, df_copy1, on="customer_id", how="inner") 

merged.to_csv("merged_customers_tickets.csv", index=False)

print(merged.head())

  customer_id            name                  email signup_date    source  \
0   CUST00005   John Gonzales      john5@mailhub.org  2024-01-06  Facebook   
1   CUST00007  Michael Bailey   michael7@mailhub.org  2024-01-08   YouTube   
2   CUST00007  Michael Bailey   michael7@mailhub.org  2024-01-08   YouTube   
3   CUST00009  Cindy Anderson                Unknown  2024-01-10    Google   
4   CUST00017      Patty Paul  patty17@inboxmail.net  2024-01-18   YouTube   

    region plan_selected marketing_opt_in   age      gender status_x  \
0    South       Premium               No  34.0       Other   active   
1  Central           Pro              Yes  60.0       Other   active   
2  Central           Pro              Yes  60.0       Other   active   
3     East       Premium               No  29.0      Female   active   
4     East           Pro               No  53.0  Non-binary   active   

   ticket_id ticket_date       issue_type resolved status_y  
0  TKT0008-1  2024-06-04            

In [ ]:
df_copy['signup_date'] = pd.to_datetime(df_copy['signup_date'], errors='coerce')
df_copy1['ticket_date'] = pd.to_datetime(df_copy1['ticket_date'], errors='coerce')

df_copy = df_copy.dropna(subset=['signup_date'])
df_copy1 = df_copy1.dropna(subset=['ticket_date'])

days_after_signup = (df_copy1['ticket_date'] - df_copy['signup_date']).dt.days

within_2_weeks = days_after_signup <= 14

print(f"Number of customers who contacted support within 2 weeks of sign-up: {within_2_weeks}")

Number of customers who contacted support within 2 weeks of sign-up: 0      False
1      False
2      False
3      False
4      False
       ...  
295    False
296    False
297    False
298    False
299    False
Length: 296, dtype: bool


In [32]:
summary = (
    merged
    .groupby(['plan_selected', 'region'])
    .agg(
        total_tickets=('ticket_id', 'count'),
        unique_customers=('customer_id', 'nunique')
    )
    .reset_index()
)

print(summary)

   plan_selected   region  total_tickets  unique_customers
0          Basic  Central              2                 1
1          Basic     East             11                 6
2          Basic    North              3                 2
3          Basic    South             14                 6
4          Basic  Unknown              2                 1
5          Basic     West             10                 4
6        Premium  Central              6                 2
7        Premium     East              1                 1
8        Premium    North              6                 2
9        Premium    South              2                 2
10       Premium     West             11                 5
11           Pro  Central             10                 5
12           Pro     East             14                 6
13           Pro    North             11                 6
14           Pro    South              3                 2
15           Pro  Unknown              3                